In [99]:
import pandas as pd
import numpy as np

In [100]:
test_df = pd.read_parquet('test.parquet')

In [101]:
X_test_list = []

for index, row in test_df.iterrows():
    time_index = pd.Index(row['dates'])  
    values = np.array(row['values'])  
    time_series = pd.Series(values, index=time_index) 
    X_test_list.append(time_series)  

X_test = pd.DataFrame(X_test_list) 

In [102]:
X_test = X_test.apply(lambda x: x.ffill(), axis=1)
X_test = X_test.apply(lambda x: x.bfill(), axis=1)

In [103]:
X_test.shape

(20000, 97)

### в тесте тоже есть полностью пустые строки, к счастью их мало, потому что там будет случайный результат

In [104]:
for column in X_test.columns:
    if X_test[column].dtype in [float, int]: 
        median_value = X_test[column].median()
        X_test[column] = X_test[column].fillna(median_value) 

In [ ]:
!pip install sktime
!pip install mlflow

In [106]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested
)

In [107]:
X_test = from_2d_array_to_nested(X_test)

In [108]:
from sktime.utils import mlflow_sktime

In [109]:
balanced_forest = mlflow_sktime.load_model(model_uri="model_forest_balanced")  

In [110]:
pred_proba = balanced_forest.predict_proba(X_test)

In [111]:
pred_proba.shape

(20000, 2)

In [112]:
test_df["score"] = pred_proba[:,1]
test_df = test_df.drop(columns=["dates", "values"])

In [113]:
test_df.to_csv('submission.csv', index=False)